In [10]:
from metropolis import metro_util
from WaveFunctions import *
from Hamiltonians import *
from WaveFunctions.simplegaussian import SimpleGaussian
from sampler import Sampler

ImportError: cannot import name 'Sampler' from 'sampler' (C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\QMB-Spesialpensum\RBM\sampler.py)

In [2]:
HO_shape = [1, 1, 1]
hard_core_r = 0.00433

config = {
    "name"        : "demo",
    "particles"   : 2,
    "dimensions"  : 3,
    
    "metropolis"  : {"name" : "importance", "step_length" : 0.01, "steps" : 1e4, "equilibrationsteps" : 1e4},
    "optimizer"   : {"name" : "grid", "start" : 0.4, "end" : 0.6, "step" : 0.05},
    
    "initial"     : {"name" : "randomUniform", "radius" : hard_core_r},
    "wavefunc"    : {"name" : "simpleGaussian", "alpha" : 0.5, "HO shape" : HO_shape},
    "hamiltonian" : {"name" : "HO", "omega" : 1, "HO shape" : HO_shape},
    
    #"waveFunc"   : {"name" : "boson-corr", "alpha" : 0.5, "HO shape" : HO_shape, "hard-core-r" : hard_core_r}
}

In [3]:
wf        = wf_util.new(config["wavefunc"])
ham       = ham_util.new(config["hamiltonian"])
run_steps = metro_util(config) # particle creation and placement is handled in here, since it needs to be done for each run

In [7]:
from numba import deferred_type

In [11]:
wf_type = deferred_type()
wf_type.define(globals()[wf.__class__.__name__].class_type.instance_type)

In [ ]:
sampler = Sampler(wf)
sampler.get()

In [ ]:
run_steps(wf, ham)

In [ ]:
particles = Particles(config["particles"], config["dimensions"])

particles.setPos(4, np.array([1,1,1]))
particles.getPos(4)

In [ ]:
initial = initial_util.new({"name" : "randomUniform", "radius" : 0.3})
initial.initialize(particles)

In [ ]:
wf = wf_util.new({"name" : "simpleGaussian", "alpha" : 0.5, "HO shape" : [1, 1, 1]})
wf.ratio(particles, 2, np.array([0, 0.5, 1]))

In [ ]:
ham.potential(particles)

In [ ]:
step_length = 0.1

In [ ]:
p1 = np.random.randint(num)
oldPos = particles.getPos(p1)

adj_dir = np.random.randint(1, dim)
adj_sign = (np.random.randint(2) - 0.5) * 2
newPos = oldPos + 0
newPos[adj_dir] += step_length * adj_sign

particles.setPos(p1, newPos)
ratio = wf.ratio(particles, p1, oldPos)

In [ ]:
np.random.randint(2)

In [ ]:
ratio

In [ ]:
num = 2
dim = 3

@njit
def metro(particles, wf):
    p1 = np.random.randint(num)
    oldPos = particles.getPos(p1)

    adj_dir = np.random.randint(1, dim)
    adj_sign = (np.random.randint(1) - 0.5) * 2
    newPos = oldPos + step_length * adj_sign

    particles.setPos(p1, newPos)
    ratio = wf.ratio(particles, p1, oldPos)
    if (np.random.random() < np.power(ratio, 2)):
        return True
    else:
        particles.setPos(p1, oldPos)
        return False

In [ ]:
metro(particles, wf)

In [ ]:
dims = 3
step = metro
equil = 100
@njit
def steps(initial, wf, ham):
    particles = Particles(num, dims)
    initial.initialize(particles)
    sampler = Sampler(particles, wf, ham)

    for i in range(equil):
        step(particles, wf)
    for i in range(100):
        accepted = step(particles, wf)
        sampler.sample(accepted)
    return sampler

In [ ]:
steps(initial, wf, ham)